In [10]:
# This file normalize dataframe by min max normalization

import pandas as pd
import numpy as np
import time
import os 
from pandas import Series, DataFrame
from tqdm.notebook import tqdm

dir_ = '../data/'
file_name = 'filter_track_20_user_100.pkl'

In [11]:
lh = pd.read_pickle(os.path.join(dir_, file_name))

In [12]:
unique_users = lh['uid'].unique()
unique_user = len(unique_users)
unique_tracks = lh['tid'].unique()
unique_track = len(unique_tracks)

print(unique_user)
print(unique_track)

945
39088


In [13]:
lh

,uid,tid,count
17,0,17,7
22,0,22,6
25,0,25,17
26,0,26,15
27,0,27,18
...,...,...,...
4406928,991,346622,4
4406949,991,365227,1
4406986,991,388562,4
4406992,991,399751,3


In [14]:
track_keys = {}

for track in unique_tracks:
    track_keys[track] = len(track_keys)
    
user_keys = {}

for user in unique_users:
    user_keys[user] = len(user_keys)

In [15]:
lh['tid']= lh['tid'].map(track_keys)
lh['uid']= lh['uid'].map(user_keys)

In [16]:
lh

,uid,tid,count
17,0,0,7
22,0,1,6
25,0,2,17
26,0,3,15
27,0,4,18
...,...,...,...
4406928,944,38979,4
4406949,944,39023,1
4406986,944,39049,4
4406992,944,39057,3


In [17]:
threshold = []
for i in tqdm(lh['uid'].unique()):
    user = lh[lh['uid'] == i]
    user = user.sort_values(by=['count'])
    t = []
    for j in range(19):
        v = user.iloc[len(user)//20*(j+1)]['count']
        t.append(v)
    threshold.append(t)
threshold

[[1, 1, 1, 1, 1, 2, 2, 2, 3, 3, 4, 4, 5, 6, 8, 9, 11, 15, 20],
 [1, 1, 1, 1, 1, 2, 2, 3, 3, 4, 5, 6, 7, 8, 10, 12, 16, 21, 30],
 [1, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 4, 4, 5, 5, 6, 8, 11, 16],
 [1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 4, 5, 6, 10],
 [1, 1, 1, 1, 2, 2, 2, 3, 4, 4, 5, 5, 6, 8, 9, 11, 16, 28, 51],
 [1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 4, 5, 6, 8, 10, 17],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 4, 4, 5],
 [1, 1, 1, 1, 1, 2, 2, 2, 2, 3, 4, 7, 8, 10, 30, 33, 35, 38, 85],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 3, 4, 5, 7],
 [1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 4, 5, 8, 13, 22],
 [1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 4, 5, 6, 8],
 [1, 1, 1, 1, 2, 2, 2, 3, 4, 4, 5, 6, 7, 8, 10, 12, 14, 18, 26],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 4, 5, 6, 9],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 3, 4, 5, 7, 10, 15, 23],
 [1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 4, 5, 7, 13],
 [1, 1

In [18]:
a = threshold[0][18]
print(a)

20


In [19]:
start = time.time()
def to_rating(df):  
    rating = 0
    for i in range(18,-1,-1):
        if df['count'] <= threshold[df['uid']][i]:
            continue
        else:
            rating = i+1
            break
#     print(df['uid'],df['count'],rating)
    return 0.25*(rating+1)
        
lh['rating'] = lh.apply(to_rating , axis = 1)
lh = lh[['uid','tid','rating','count']]

end = time.time()
print(end - start)

356.8600001335144


In [20]:
lh

,uid,tid,rating,count
17,0,0,3.75,7
22,0,1,3.50,6
25,0,2,4.75,17
26,0,3,4.50,15
27,0,4,4.75,18
...,...,...,...,...
4406928,944,38979,3.00,4
4406949,944,39023,0.25,1
4406986,944,39049,3.00,4
4406992,944,39057,2.25,3


In [21]:
lh.to_pickle(os.path.join(dir_, 'normalized_to_rating_' + file_name))

In [14]:
len(lh[lh['rating']>1])

1707339

In [16]:
lc = lh[lh['rating']>0]
print(len(lc))
lc2 = lc[lc['count']>4]
print(len(lc2)/len(lc))

3079630
0.24044446897841623
